In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 설정
BASE_URL = 'https://m.dcinside.com'
GALLERY_ID = 'programming'  # 예시: 프로그래밍 갤러리 (네가 원하는 걸로 바꿔)
MAX_POSTS = 100
MAX_COMMENTS_PER_POST = 50

# 결과 저장
posts = []

# Step 1: 인기글 리스트 가져오기
def get_popular_posts(gallery_id):
    url = f'{BASE_URL}/board/{gallery_id}?recommend=1'  # 개념글 목록
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    links = soup.select('tr.ub-content > td.gall_tit > a')
    post_links = [BASE_URL + link['href'] for link in links]
    return post_links

# Step 2: 게시글 본문 + 댓글 가져오기
def get_post_details(post_url):
    res = requests.get(post_url)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    try:
        title = soup.select_one('div.ub-content > div.title_subject').text.strip()
    except:
        title = ""

    try:
        body = soup.select_one('div.write_div').text.strip()
    except:
        body = ""

    try:
        like_count = int(soup.select_one('span.up_num').text.strip())
    except:
        like_count = 0

    try:
        comment_count = int(soup.select_one('span.cmt_count').text.strip().replace('댓글', '').strip())
    except:
        comment_count = 0

    try:
        time_tag = soup.select_one('div.gall_date')['title']
    except:
        time_tag = ""

    post_id = post_url.split('/')[-1]
    
    comments = get_comments(post_id)
    
    return {
        'post_id': post_id,
        'title': title,
        'body': body,
        'like_count': like_count,
        'comment_count': comment_count,
        'time': time_tag,
        'link': post_url,
        'comments': comments
    }

# Step 3: 댓글 가져오기
def get_comments(post_id):
    comments = []
    page = 1

    while len(comments) < MAX_COMMENTS_PER_POST:
        comment_url = f"https://m.dcinside.com/board/comment/{post_id}?page={page}"
        res = requests.get(comment_url)
        soup = BeautifulSoup(res.text, 'html.parser')
        comment_list = soup.select('ul.comment_box > li')
        
        if not comment_list:
            break  # 더 이상 댓글 없음
        
        for comment in comment_list:
            content_tag = comment.select_one('p.usertxt')
            if content_tag:
                content = content_tag.text.strip()
                comments.append(content)
            
            if len(comments) >= MAX_COMMENTS_PER_POST:
                break
        
        page += 1
        time.sleep(0.1)  # 서버 부하 줄이기

    return comments

# 메인 크롤링
def crawl_dc_popular(gallery_id):
    post_links = get_popular_posts(gallery_id)
    print(f"Found {len(post_links)} popular posts.")
    
    for i, link in enumerate(post_links[:MAX_POSTS]):
        print(f"[{i+1}/{MAX_POSTS}] 크롤링 중: {link}")
        post = get_post_details(link)
        posts.append(post)
        time.sleep(0.5)  # 서버 부하 줄이기

    # CSV 저장
    df = pd.DataFrame(posts)
    df.to_csv(f'dc_{gallery_id}_popular.csv', index=False, encoding='utf-8-sig')
    print("CSV 저장 완료")

# 실행
if __name__ == "__main__":
    crawl_dc_popular(GALLERY_ID)


In [11]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

# 설정
BASE_URL = "https://gall.dcinside.com"
SILBE_URL = f"{BASE_URL}/board/lists?id=dcbest"
MAX_POSTS = 50
MAX_COMMENTS_PER_POST = 50
POSTS_PER_PAGE = 20

# 세션 생성
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Referer": "https://gall.dcinside.com/"
})

results = []
crawled_post_ids = set()

# 글 링크 가져오기 (공지/고정글 제외)
def get_post_links(page_num):
    url = f"{SILBE_URL}&page={page_num}"
    res = session.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    links = []
    
    posts = soup.select('tr.ub-content.us-post')  # 공지 제외하고 진짜 글만
    for post in posts:
        link_tag = post.select_one('td.gall_tit a')
        if link_tag:
            href = link_tag.get("href")
            if href and '/board/view/' in href:
                if href.startswith('http'):
                    full_link = href
                else:
                    full_link = BASE_URL + href
                links.append(full_link)
    return links

# 댓글 가져오기 (댓글 전용 API 호출)
def get_comments(post_id):
    comments = []
    page = 1
    
    try:
        # 게시글 페이지에서 필요한 정보 추출
        post_url = f"{BASE_URL}/board/view/?id=dcbest&no={post_id}"
        res = session.get(post_url)
        soup = BeautifulSoup(res.text, "html.parser")
        
        # e_s_n_o 값을 찾기 (페이지 소스에서 이 값을 추출해야 함)
        script_tags = soup.find_all('script')
        e_s_n_o = None
        for script in script_tags:
            if script.string and 'e_s_n_o' in str(script.string):
                match = re.search(r'e_s_n_o\s*=\s*"([^"]+)"', str(script.string))
                if match:
                    e_s_n_o = match.group(1)
                    break
        
        if not e_s_n_o:
            print(f"게시글 {post_id}에서 e_s_n_o를 찾을 수 없습니다.")
            return comments
            
        while len(comments) < MAX_COMMENTS_PER_POST:
            # XHR 요청과 동일한 헤더 설정
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
                "Referer": post_url,
                "X-Requested-With": "XMLHttpRequest",
                "Accept": "application/json, text/javascript, */*; q=0.01"
            }
            
            # 댓글 API 요청 (새로운 형식)
            comment_url = f"{BASE_URL}/board/comment/"
            params = {
                "id": "dcbest",
                "no": post_id,
                "cmt_id": "dcbest",
                "cmt_no": post_id,
                "e_s_n_o": e_s_n_o,
                "comment_page": page
            }
            
            res = session.get(comment_url, params=params, headers=headers)
            
            # JSON 응답 처리 시도
            try:
                json_data = res.json()
                if 'comments' in json_data:
                    for comment in json_data['comments']:
                        if 'comment' in comment:
                            comments.append(comment['comment'])
                        if len(comments) >= MAX_COMMENTS_PER_POST:
                            break
                else:
                    # JSON 형식이지만 comments 키가 없는 경우
                    break
            except:
                # JSON 파싱 실패 - HTML 방식으로 시도
                soup = BeautifulSoup(res.text, "html.parser")
                comment_tags = soup.select('.comment_box .comment_text')
                
                if not comment_tags:
                    break
                    
                for comment_tag in comment_tags:
                    text = comment_tag.get_text(strip=True)
                    if text:
                        comments.append(text)
                    if len(comments) >= MAX_COMMENTS_PER_POST:
                        break
            
            # 다음 페이지가 없거나 충분한 댓글을 모았으면 종료
            if len(comments) < (page * 20) or len(comments) >= MAX_COMMENTS_PER_POST:
                break
                
            page += 1
            time.sleep(0.2)  # 조금 더 긴 대기 시간
            
    except Exception as e:
        print(f"댓글 수집 오류 (게시글 {post_id}): {e}")
        
    return comments


# 글 본문 + 댓글 가져오기
def get_post_details(post_url):
    res = session.get(post_url)
    soup = BeautifulSoup(res.text, "html.parser")
    
    try:
        title = soup.select_one("span.title_subject").text.strip()
    except:
        title = ""

    try:
        content = soup.select_one("div.write_div").text.strip()
    except:
        content = "본문 없음"

    try:
        post_id = post_url.split('no=')[1].split('&')[0]
    except:
        post_id = ""

    comments = get_comments(post_id)

    return {
        "post_id": post_id,
        "title": title,
        "link": post_url,
        "content": content,
        "comments": comments
    }

# 메인 크롤링 함수
def crawl_silbe():
    page = 1
    crawled_posts = 0

    while crawled_posts < MAX_POSTS:
        print(f"페이지 {page} 크롤링 중...")
        post_links = get_post_links(page)

        for post_link in post_links:
            if crawled_posts >= MAX_POSTS:
                break

            post_id = post_link.split('no=')[1].split('&')[0]
            if post_id in crawled_post_ids:
                continue  # 이미 크롤링한 글이면 스킵

            try:
                post = get_post_details(post_link)
                results.append(post)
                crawled_post_ids.add(post_id)
                crawled_posts += 1
                print(f"[{crawled_posts}] {post['title']} 크롤링 완료")
                time.sleep(0.5)
            except Exception as e:
                print(f"에러 발생 (패스): {e}")
                continue
        
        page += 1
        time.sleep(1)

    # 결과 저장
    df = pd.DataFrame(results)
    df.to_csv('dcinside_silbe_100posts.csv', index=False, encoding='utf-8-sig')
    print("\n✅ CSV 파일 저장 완료: dcinside_silbe_100posts.csv")

if __name__ == "__main__":
    crawl_silbe()


페이지 1 크롤링 중...
게시글 326081에서 e_s_n_o를 찾을 수 없습니다.
[1] 오늘 아침에 일어났던 청주시 고등학교 사건에 대한 기사 크롤링 완료
게시글 326078에서 e_s_n_o를 찾을 수 없습니다.
[2] 준비란 물량까지 동났다...어르신들 공략한 매력적 제안 크롤링 완료
게시글 326077에서 e_s_n_o를 찾을 수 없습니다.
[3] 첫해외여행&혼여 오사카-교토 5박6일 후기 크롤링 완료
게시글 326074에서 e_s_n_o를 찾을 수 없습니다.
[4] 대구 북구 노곡동 함지산서 산불 발생…인근 주민·등산객 주의 크롤링 완료
게시글 326072에서 e_s_n_o를 찾을 수 없습니다.
[5] 전장연, 출근길 혜화역 지하철 시위… 또 강제 퇴거당해 크롤링 완료
게시글 326071에서 e_s_n_o를 찾을 수 없습니다.
[6] 공포의 인도 남성인권 운동 크롤링 완료
게시글 326069에서 e_s_n_o를 찾을 수 없습니다.
[7] 전라도 군산시, 백종원 더본코리아에 70억 예산 투입 크롤링 완료
게시글 326068에서 e_s_n_o를 찾을 수 없습니다.
[8] 마법과 마나의 세계에서 존재할리 없는 쿠노이치.1 크롤링 완료
게시글 326066에서 e_s_n_o를 찾을 수 없습니다.
[9] 김재연 "여가부 장관 부총리로 격상, 비동간죄·차금법 통과, 탈원전" 크롤링 완료
게시글 326065에서 e_s_n_o를 찾을 수 없습니다.
[10] 씹덕 행사장에서 버튜버를 알아본 사람 크롤링 완료
게시글 326063에서 e_s_n_o를 찾을 수 없습니다.
[11] 싱글벙글 스타크래프트 IP경쟁 근황.jpg 크롤링 완료
게시글 326062에서 e_s_n_o를 찾을 수 없습니다.
[12] "닦달하면 환불 없어" '황당'문자.."돈 떼이고 협박당하나" 크롤링 완료
게시글 326060에서 e_s_n_o를 찾을 수 없습니다.
[13] 한동훈 한화 이글스 뭐냐??????ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 크롤링 완료
게시글 326059에서 e_s_n_o를 찾을 수 없습니다.


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import random
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# 설정
BASE_URL = "https://gall.dcinside.com"
SILBE_URL = f"{BASE_URL}/board/lists?id=dcbest"
MAX_POSTS = 50
MAX_COMMENTS_PER_POST = 50
POSTS_PER_PAGE = 20

# User-Agent 목록
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"
]

# 세션 생성
session = requests.Session()

# 세션 헤더 랜덤 설정 함수
def update_session_headers():
    session.headers.update({
        "User-Agent": random.choice(USER_AGENTS),
        "Referer": "https://gall.dcinside.com/"
    })

update_session_headers()

results = []
crawled_post_ids = set()

# 글 링크 가져오기 (공지/고정글 제외)
def get_post_links(page_num):
    update_session_headers()
    url = f"{SILBE_URL}&page={page_num}"
    try:
        res = session.get(url)
        soup = BeautifulSoup(res.text, "html.parser")
        posts = soup.select('tr.ub-content.us-post')
        links = []
        for post in posts:
            link_tag = post.select_one('td.gall_tit a')
            if link_tag:
                href = link_tag.get("href")
                if href and '/board/view/' in href:
                    full_link = href if href.startswith('http') else BASE_URL + href
                    # 게시글 ID 추출
                    try:
                        post_id = href.split('no=')[1].split('&')[0]
                    except:
                        post_id = ""
                    # 작성시간, 추천수 추출
                    try:
                        write_time = post.select_one('td.gall_date').text.strip()
                    except:
                        write_time = ""
                    try:
                        recommend = post.select_one('td.gall_recommend').text.strip()
                    except:
                        recommend = ""
                    links.append({
                        "link": full_link,
                        "post_id": post_id,
                        "write_time": write_time,
                        "recommend": recommend
                    })
        return links
    except Exception as e:
        print(f"글 목록 가져오기 오류: {e}")
        return []


# 댓글 가져오기 (API 요청 방식)
def get_comments_api(post_id):
    comments = []
    page = 1
    
    try:
        # 게시글 페이지에서 필요한 정보 추출
        update_session_headers()
        post_url = f"{BASE_URL}/board/view/?id=dcbest&no={post_id}"
        res = session.get(post_url)
        soup = BeautifulSoup(res.text, "html.parser")
        
        # e_s_n_o 값을 찾기 (페이지 소스에서 이 값을 추출해야 함)
        script_tags = soup.find_all('script')
        e_s_n_o = None
        for script in script_tags:
            if script.string and 'e_s_n_o' in str(script.string):
                match = re.search(r'e_s_n_o\s*=\s*"([^"]+)"', str(script.string))
                if match:
                    e_s_n_o = match.group(1)
                    break
        
        if not e_s_n_o:
            print(f"게시글 {post_id}에서 e_s_n_o를 찾을 수 없습니다. 다른 방식으로 시도합니다.")
            return get_comments_html(post_id, soup)
            
        while len(comments) < MAX_COMMENTS_PER_POST:
            update_session_headers()
            
            # 댓글 API 요청
            comment_url = f"{BASE_URL}/board/comment/"
            params = {
                "id": "dcbest",
                "no": post_id,
                "cmt_id": "dcbest",
                "cmt_no": post_id,
                "e_s_n_o": e_s_n_o,
                "comment_page": page
            }
            
            headers = session.headers.copy()
            headers.update({
                "X-Requested-With": "XMLHttpRequest",
                "Accept": "application/json, text/javascript, */*; q=0.01",
                "Referer": post_url
            })
            
            res = session.get(comment_url, params=params, headers=headers)
            
            # JSON 응답 처리 시도
            try:
                json_data = res.json()
                if 'comments' in json_data:
                    found_comments = False
                    for comment in json_data['comments']:
                        if 'comment' in comment:
                            comments.append(comment['comment'])
                            found_comments = True
                        if len(comments) >= MAX_COMMENTS_PER_POST:
                            break
                    if not found_comments:
                        break
                else:
                    # JSON 형식이지만 comments 키가 없는 경우
                    break
            except:
                # JSON 파싱 실패 시 HTML 방식으로 전환
                print(f"JSON 파싱 실패. HTML 방식으로 전환합니다.")
                html_comments = get_comments_html(post_id, None)
                comments.extend(html_comments)
                break
            
            # 다음 페이지가 없거나 충분한 댓글을 모았으면 종료
            if len(comments) < (page * 20) or len(comments) >= MAX_COMMENTS_PER_POST:
                break
                
            page += 1
            time.sleep(random.uniform(0.5, 1.0))  # 랜덤 지연
            
    except Exception as e:
        print(f"API 댓글 수집 오류 (게시글 {post_id}): {e}")
        # 오류 발생 시 HTML 방식으로 시도
        html_comments = get_comments_html(post_id, None)
        comments.extend(html_comments)
        
    return comments

# HTML 파싱 방식으로 댓글 가져오기 (대체 방식)
def get_comments_html(post_id, existing_soup=None):
    comments = []
    
    try:
        if existing_soup is None:
            update_session_headers()
            post_url = f"{BASE_URL}/board/view/?id=dcbest&no={post_id}"
            res = session.get(post_url)
            soup = BeautifulSoup(res.text, "html.parser")
        else:
            soup = existing_soup
        
        # 여러 가능한 CSS 선택자로 시도
        selectors = [
            'div.comment_box div.usertxt',
            'div.comment_box span.comment_text',
            'div.reply_box div.reply_text',
            'p.usertxt',
            'div.comment_view p.inner_info'
        ]
        
        for selector in selectors:
            comment_tags = soup.select(selector)
            if comment_tags:
                for comment_tag in comment_tags:
                    text = comment_tag.get_text(strip=True)
                    if text:
                        comments.append(text)
                    if len(comments) >= MAX_COMMENTS_PER_POST:
                        break
                break  # 성공한 선택자가 있으면 루프 종료
                
    except Exception as e:
        print(f"HTML 댓글 수집 오류 (게시글 {post_id}): {e}")
    
    return comments

# Selenium을 이용한 댓글 크롤링 (최후의 수단)
def get_comments_selenium(post_id):
    comments = []
    
    # Chrome 옵션 설정
    options = Options()
    options.add_argument('--headless')  # 헤드리스 모드
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument(f'user-agent={random.choice(USER_AGENTS)}')
    
    # 웹드라이버 초기화
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    try:
        # 게시글 페이지 접속
        post_url = f"{BASE_URL}/board/view/?id=dcbest&no={post_id}"
        driver.get(post_url)
        time.sleep(2)  # 페이지 로딩 대기
        
        # 댓글 영역의 여러 가능한 선택자
        selectors = [
            ".comment_box .comment_text",
            ".comment_box .usertxt",
            ".reply_box .reply_text"
        ]
        
        for selector in selectors:
            comment_elements = driver.find_elements(By.CSS_SELECTOR, selector)
            if comment_elements:
                for element in comment_elements:
                    text = element.text.strip()
                    if text:
                        comments.append(text)
                    if len(comments) >= MAX_COMMENTS_PER_POST:
                        break
                break  # 성공한 선택자가 있으면 루프 종료
                
    except Exception as e:
        print(f"Selenium 댓글 수집 오류 (게시글 {post_id}): {e}")
        
    finally:
        driver.quit()
        
    return comments

def get_comments(post_id):
    # 무조건 Selenium 사용
    return get_comments_selenium(post_id)

def get_comments_selenium(post_id):
    comments = []
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument(f'user-agent={random.choice(USER_AGENTS)}')

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    try:
        post_url = f"{BASE_URL}/board/view/?id=dcbest&no={post_id}"
        driver.get(post_url)
        time.sleep(2)  # 충분한 로딩 대기

        # 댓글이 여러 페이지에 걸쳐 있을 수 있으므로 반복
        while True:
            # 댓글 텍스트 추출
            comment_elements = driver.find_elements(By.CLASS_NAME, "usertxt")
            for elem in comment_elements:
                text = elem.text.strip()
                if text and text not in comments:
                    comments.append(text)
                if len(comments) >= MAX_COMMENTS_PER_POST:
                    break
            if len(comments) >= MAX_COMMENTS_PER_POST:
                break

            # "더보기" 버튼 또는 다음 페이지 버튼 찾기 (없으면 종료)
            try:
                more_btn = driver.find_element(By.CSS_SELECTOR, ".comment_more_btn")
                if more_btn.is_displayed():
                    more_btn.click()
                    time.sleep(1)
                else:
                    break
            except:
                break  # 더 이상 버튼이 없으면 종료

    except Exception as e:
        print(f"Selenium 댓글 수집 오류 (게시글 {post_id}): {e}")
    finally:
        driver.quit()
    return comments


# 글 본문 + 댓글 가져오기
def get_post_details(post_url):
    update_session_headers()
    res = session.get(post_url)
    soup = BeautifulSoup(res.text, "html.parser")
    
    try:
        title = soup.select_one("span.title_subject").text.strip()
    except:
        title = "제목 없음"

    try:
        content = soup.select_one("div.write_div").text.strip()
    except:
        content = "본문 없음"

    try:
        post_id = post_url.split('no=')[1].split('&')[0]
    except:
        post_id = ""

    comments = get_comments(post_id)

    return {
        "post_id": post_id,
        "title": title,
        "link": post_url,
        "content": content,
        "comments": comments
    }

# 메인 크롤링 함수
def crawl_silbe():
    USE_SELENIUM = False  # Selenium 사용 여부 설정
    page = 1
    crawled_posts = 0

    while crawled_posts < MAX_POSTS:
        print(f"페이지 {page} 크롤링 중...")
        post_links = get_post_links(page)

        if not post_links:
            print(f"페이지 {page}에서 글을 찾을 수 없습니다. 다음 페이지로 넘어갑니다.")
            page += 1
            time.sleep(random.uniform(1.0, 2.0))
            continue

        for post_link in post_links:
            if crawled_posts >= MAX_POSTS:
                break

            try:
                post_id = post_link.split('no=')[1].split('&')[0]
            except:
                print(f"링크에서 게시글 ID를 추출할 수 없습니다: {post_link}")
                continue
                
            if post_id in crawled_post_ids:
                continue  # 이미 크롤링한 글이면 스킵

            try:
                print(f"게시글 {post_id} 크롤링 중...")
                post = get_post_details(post_link)
                results.append(post)
                crawled_post_ids.add(post_id)
                crawled_posts += 1
                print(f"[{crawled_posts}/{MAX_POSTS}] {post['title']} 크롤링 완료 (댓글 {len(post['comments'])}개)")
                time.sleep(random.uniform(0.8, 1.5))  # 랜덤 지연
            except Exception as e:
                print(f"게시글 크롤링 오류 (패스): {e}")
                continue
        
        page += 1
        time.sleep(random.uniform(1.5, 3.0))  # 페이지 간 랜덤 지연

    # 결과 저장
    df = pd.DataFrame(results)
    df.to_csv('dcinside_silbe_posts.csv', index=False, encoding='utf-8-sig')
    print("\n✅ CSV 파일 저장 완료: dcinside_silbe_posts.csv")

if __name__ == "__main__":
    crawl_silbe()


페이지 1 크롤링 중...
게시글 326084 크롤링 중...
[1/50] SKT 쓰는 임원, 빨리 바꿔라" 대기업도 '비상'…커지는 불안 크롤링 완료 (댓글 14개)
게시글 326083 크롤링 중...
[2/50] 롯데 우승까지 유튜브한다는 아로치카 근황..jpg 크롤링 완료 (댓글 17개)
게시글 326081 크롤링 중...
[3/50] 오늘 아침에 일어났던 청주시 고등학교 사건에 대한 기사 크롤링 완료 (댓글 50개)
게시글 326078 크롤링 중...
[4/50] 준비란 물량까지 동났다...어르신들 공략한 매력적 제안 크롤링 완료 (댓글 50개)
게시글 326077 크롤링 중...
[5/50] 첫해외여행&혼여 오사카-교토 5박6일 후기 크롤링 완료 (댓글 4개)
게시글 326074 크롤링 중...
[6/50] 대구 북구 노곡동 함지산서 산불 발생…인근 주민·등산객 주의 크롤링 완료 (댓글 50개)
게시글 326072 크롤링 중...
[7/50] 전장연, 출근길 혜화역 지하철 시위… 또 강제 퇴거당해 크롤링 완료 (댓글 50개)
게시글 326071 크롤링 중...
[8/50] 공포의 인도 남성인권 운동 크롤링 완료 (댓글 50개)
게시글 326069 크롤링 중...
[9/50] 전라도 군산시, 백종원 더본코리아에 70억 예산 투입 크롤링 완료 (댓글 50개)
게시글 326068 크롤링 중...
[10/50] 마법과 마나의 세계에서 존재할리 없는 쿠노이치.1 크롤링 완료 (댓글 35개)
게시글 326066 크롤링 중...
[11/50] 김재연 "여가부 장관 부총리로 격상, 비동간죄·차금법 통과, 탈원전" 크롤링 완료 (댓글 50개)
게시글 326065 크롤링 중...
[12/50] 씹덕 행사장에서 버튜버를 알아본 사람 크롤링 완료 (댓글 50개)
게시글 326063 크롤링 중...
[13/50] 싱글벙글 스타크래프트 IP경쟁 근황.jpg 크롤링 완료 (댓글 50개)
게시글 326062 크롤링 중...
[14/50] "닦달하면 환불 없어" '황당'문자.."돈

In [18]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import random
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# 설정
BASE_URL = "https://gall.dcinside.com"
SILBE_URL = f"{BASE_URL}/board/lists?id=dcbest"
MAX_POSTS = 50
MAX_COMMENTS_PER_POST = 50

USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"
]

session = requests.Session()

def update_session_headers():
    session.headers.update({
        "User-Agent": random.choice(USER_AGENTS),
        "Referer": BASE_URL + "/"
    })

update_session_headers()

results = []
crawled_post_ids = set()

# 글 목록에서 링크, post_id, 작성시간, 추천수 추출
def get_post_links(page_num):
    update_session_headers()
    url = f"{SILBE_URL}&page={page_num}"
    try:
        res = session.get(url)
        soup = BeautifulSoup(res.text, "html.parser")
        links = []
        posts = soup.select('tr.ub-content.us-post')
        for post in posts:
            link_tag = post.select_one('td.gall_tit a')
            if link_tag:
                href = link_tag.get("href")
                if href and '/board/view/' in href:
                    full_link = href if href.startswith('http') else BASE_URL + href
                    try:
                        post_id = href.split('no=')[1].split('&')[0]
                    except:
                        post_id = ""
                    try:
                        write_time = post.select_one('td.gall_date').text.strip()
                    except:
                        write_time = ""
                    try:
                        recommend = post.select_one('td.gall_recommend').text.strip()
                    except:
                        recommend = ""
                    links.append({
                        "link": full_link,
                        "post_id": post_id,
                        "write_time": write_time,
                        "recommend": recommend
                    })
        return links
    except Exception as e:
        print(f"글 목록 가져오기 오류: {e}")
        return []

# Selenium으로 댓글 크롤링
def get_comments_selenium(post_id):
    comments = []
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument(f'user-agent={random.choice(USER_AGENTS)}')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    try:
        post_url = f"{BASE_URL}/board/view/?id=dcbest&no={post_id}"
        driver.get(post_url)
        time.sleep(2)
        # 댓글 여러 페이지 지원: 더보기 버튼 클릭 반복
        while True:
            comment_elements = driver.find_elements(By.CLASS_NAME, "usertxt")
            for elem in comment_elements:
                text = elem.text.strip()
                if text and text not in comments:
                    comments.append(text)
                if len(comments) >= MAX_COMMENTS_PER_POST:
                    break
            if len(comments) >= MAX_COMMENTS_PER_POST:
                break
            try:
                more_btn = driver.find_element(By.CSS_SELECTOR, ".comment_more_btn")
                if more_btn.is_displayed():
                    more_btn.click()
                    time.sleep(1)
                else:
                    break
            except:
                break
    except Exception as e:
        print(f"Selenium 댓글 수집 오류 (게시글 {post_id}): {e}")
    finally:
        driver.quit()
    return comments

# 글 본문 + 댓글 + 작성시간 + 추천수
def get_post_details(post_info):
    post_url = post_info['link']
    post_id = post_info['post_id']
    write_time = post_info['write_time']
    recommend = post_info['recommend']
    update_session_headers()
    res = session.get(post_url)
    soup = BeautifulSoup(res.text, "html.parser")
    try:
        title = soup.select_one("span.title_subject").text.strip()
    except:
        title = "제목 없음"
    try:
        content = soup.select_one("div.write_div").text.strip()
    except:
        content = "본문 없음"
    comments = get_comments_selenium(post_id)
    return {
        "post_id": post_id,
        "title": title,
        "link": post_url,
        "content": content,
        "write_time": write_time,
        "recommend": recommend,
        "comments": comments
    }

def crawl_silbe():
    page = 1
    crawled_posts = 0
    while crawled_posts < MAX_POSTS:
        print(f"페이지 {page} 크롤링 중...")
        post_links = get_post_links(page)
        if not post_links:
            print(f"페이지 {page}에서 글을 찾을 수 없습니다. 다음 페이지로 넘어갑니다.")
            page += 1
            time.sleep(random.uniform(1.0, 2.0))
            continue
        for post_info in post_links:
            if crawled_posts >= MAX_POSTS:
                break
            post_id = post_info['post_id']
            if post_id in crawled_post_ids:
                continue
            try:
                print(f"게시글 {post_id} 크롤링 중...")
                post = get_post_details(post_info)
                results.append(post)
                crawled_post_ids.add(post_id)
                crawled_posts += 1
                print(f"[{crawled_posts}/{MAX_POSTS}] {post['title']} 크롤링 완료 (댓글 {len(post['comments'])}개)")
                time.sleep(random.uniform(0.8, 1.5))
            except Exception as e:
                print(f"게시글 크롤링 오류 (패스): {e}")
                continue
        page += 1
        time.sleep(random.uniform(1.5, 3.0))
    df = pd.DataFrame(results)
    df.to_csv('dcinside_silbe_posts.csv', index=False, encoding='utf-8-sig')
    print("\n✅ CSV 파일 저장 완료: dcinside_silbe_posts.csv")

if __name__ == "__main__":
    crawl_silbe()


페이지 1 크롤링 중...
게시글 326253 크롤링 중...
[1/50] 전세계에 난리난 AI 답변 크롤링 완료 (댓글 5개)
게시글 326251 크롤링 중...
[2/50] 왜노자 일본 스위치2 체험회 갔다온 후기 크롤링 완료 (댓글 44개)
게시글 326249 크롤링 중...
[3/50] 6시에 저녁식사하는 한국이 신기하다는 서양인들 크롤링 완료 (댓글 44개)
게시글 326247 크롤링 중...
[4/50] 도쿄 3박 4일로 여행아다 뗌. 3일차 (1) 크롤링 완료 (댓글 13개)
게시글 326246 크롤링 중...
[5/50] 신문고답변) 서울가락몰옥토버페스타 위법 인정 크롤링 완료 (댓글 24개)
게시글 326241 크롤링 중...
[6/50] 퍼거슨이 박지성에게 쓴 편지.....jpg 크롤링 완료 (댓글 50개)
게시글 326239 크롤링 중...
[7/50] ④ 9박 10일 일본 배낭여행기 4일차 -1,2 (모리오카,아키타,4월18일) 크롤링 완료 (댓글 7개)
게시글 326237 크롤링 중...
[8/50] 정동원 중2병 완치시켜준 장민호 크롤링 완료 (댓글 50개)
게시글 326235 크롤링 중...
[9/50] 백종원이 가위질 솔루션을 해준 이유 크롤링 완료 (댓글 50개)
게시글 326234 크롤링 중...
[10/50] 싱글벙글 스페이드 에이스 카드가 유독 특별한 이유.jpg 크롤링 완료 (댓글 46개)
게시글 326232 크롤링 중...
[11/50] 빨대로 빨아먹는 비빔면이 있다? 크롤링 완료 (댓글 50개)
게시글 326230 크롤링 중...
[12/50] 한복은 별로 입고 싶지 않다는 탈북녀 크롤링 완료 (댓글 50개)
게시글 326228 크롤링 중...
[13/50] 전광훈 후보님 공약 2탄 26~50 크롤링 완료 (댓글 50개)
게시글 326226 크롤링 중...
[14/50] 중국 춘추전국시대 보물 수준 크롤링 완료 (댓글 50개)
게시글 326224 크롤링 중...
[15/50] 응급실에 실려간 형 크롤링 완료 (댓글 50

In [20]:
import pandas as pd
import requests
import ast
import nltk
from collections import Counter
from konlpy.tag import Okt
import time

# nltk 데이터 다운로드 (최초 1번)
nltk.download('punkt')

# 1. 파일 로드
df = pd.read_csv('final_classified_posts.csv')

# comments 컬럼 파싱
def parse_comments(comment_str):
    try:
        return ast.literal_eval(comment_str)
    except:
        return []

df['comments'] = df['comments'].apply(parse_comments)

# 2. 주제별 Top 3 글 추리기
def get_top3_by_topic(df):
    topic_to_posts = {}

    for idx, row in df.iterrows():
        topics = [t.strip() for t in row['classified_topics'].split(',')]
        for topic in topics:
            if topic not in topic_to_posts:
                topic_to_posts[topic] = []
            topic_to_posts[topic].append((row['recommend'], row['title'], row['content']))
    
    # 추천수 기준 Top 3만
    for topic in topic_to_posts:
        topic_to_posts[topic] = sorted(topic_to_posts[topic], key=lambda x: x[0], reverse=True)[:3]

    return topic_to_posts

top3_by_topic = get_top3_by_topic(df)

# 3. 핫 키워드 추출
okt = Okt()

def extract_keywords(texts, top_n=10):
    all_nouns = []
    for text in texts:
        nouns = okt.nouns(text)
        all_nouns.extend(nouns)
    
    counter = Counter(all_nouns)
    most_common = counter.most_common(top_n)
    return [word for word, _ in most_common]

# 모든 글+댓글 합쳐서 키워드 추출
all_texts = df['content'].tolist() + [" ".join(comments) for comments in df['comments'].tolist()]
hot_keywords = extract_keywords(all_texts)

# 4. 감정 비율 계산
sentiment_counts = df['classified_sentiment'].value_counts(normalize=True) * 100

# 5. 전체 민심 종합 요약 (Gemma 호출)
API_URL = "http://localhost:1234/v1/chat/completions"
MODEL_NAME = "gemma:3-12b-instruct"

SYSTEM_PROMPT_SUMMARY = """당신은 글과 댓글을 분석해서 커뮤니티의 전체적인 분위기를 요약하는 전문가입니다.
다음 텍스트를 읽고, 현재 커뮤니티의 민심과 주요 분위기를 리포트로 작성해주세요요."""

HEADERS = {
    "Content-Type": "application/json"
}

def summarize_trend(text):
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT_SUMMARY},
            {"role": "user", "content": text}
        ],
        "stream": False
    }
    try:
        response = requests.post(API_URL, headers=HEADERS, json=payload)
        response.raise_for_status()
        result = response.json()
        return result['choices'][0]['message']['content'].strip()
    except Exception as e:
        print(f"요약 실패: {e}")
        return "요약 실패"

combined_text_for_summary = " ".join(df['content'].tolist())
overall_summary = summarize_trend(combined_text_for_summary)

# 6. Markdown 리포트 생성
md = "# 🔥 오늘의 커뮤니티 민심 리포트\n\n"

md += "## 📰 급상승 키워드 Top 10\n"
for kw in hot_keywords:
    md += f"- {kw}\n"

md += "\n## 🎯 주제별 인기글 요약\n"
for topic, posts in top3_by_topic.items():
    md += f"### [{topic}]\n"
    for _, title, content in posts:
        md += f"- **{title}**: {content[:100]}...\n"
    md += "\n"

md += "## ❤️ 감정 분석 결과\n"
for sentiment, percent in sentiment_counts.items():
    md += f"- {sentiment}: {percent:.1f}%\n"

md += "\n## 🧠 전체 민심 요약\n"
md += f"> {overall_summary}\n"

# 7. Markdown 저장
with open("community_report.md", "w", encoding="utf-8") as f:
    f.write(md)

print("\n✅ 리포트 작성 완료: community_report.md 파일 생성됨!")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MSI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


요약 실패: 400 Client Error: Bad Request for url: http://localhost:1234/v1/chat/completions

✅ 리포트 작성 완료: community_report.md 파일 생성됨!
